This program give information on how much one will need to retire.

add inflation
add tax estimate
add statistics - monte carlo analysis
add graphics

answers question - when will a person run out of money
                 - how much will they need to save not so they won't run out of money
                 - how long will they have to delay retirement so they won't run out of money
                 - how much will they will have to reduce their need so they won't run out of money
                
                


In [2]:
import pandas as pd
import numpy as np
import time

In [18]:
###################################################
# METHODS
###################################################

#inputs all of the needed information and return a list
def var_input():
    
    variables = []
    variable_names = []
    
    #time to retirement
    years_to_retire = int(input('In how many years do you plan to retire? '))
    
    #amount needed at retirement
    how_much_needed_at_retirement = float(input('How much per year will you need at retirement? '))
    
    #current retirement savings
    starting_principle = float(input('How much have you saved for retirement right now? [in thousands]'))
        
    #income after retirment
    how_much_ss_at_retirement = float(input('How much Social Security benefits per year will you get at retirement? '))
    how_much_other_passive_income = float(input('How much other passive income per year will you get at retirement? '))
    
    #return on retirement savings
    investment_apr = float(input('What is the annualized percent return on your retirement saving before retiring? '))

    #life expectance after retirement
    yrs_to_death = input('How long will you need retirement income? [press enter if you want 20 yrs.]')
    if yrs_to_death:
        yrs_to_death = float(yrs_to_death)
    else:
        yrs_to_death = float(20)
        print('Based on your input you will need your retirement income for ', yrs_to_death, 'years.')
    
    #contribution to retirement savings before retiring    
    additional_principle = float(input('How much will you be adding to your retirement savings per year before retiring? '))

    #inflation rate
    inflation = input('What is the inflation rate per year? [press enter if you want 3%.]')
    if inflation:
        inflation = float(inflation)
    else:
        inflation = float(3)
        print('Based on your input the inflation rate is', inflation, '%.')

    #tax rate
    tax_rate = input('Based on what you will need, what will be your tax rate? [press enter if you want to assume 25% for both state and federal taxes.]')
    if tax_rate:
        tax_rate = float(tax_rate)
    else:
        tax_rate = float(25)
        print('Based on your input your overall income tax rate is', tax_rate, '%.')
        
    final_principle = float(0)
    
    variables = [years_to_retire, how_much_needed_at_retirement, starting_principle,
                 how_much_ss_at_retirement, how_much_other_passive_income, investment_apr,
                 yrs_to_death, additional_principle, inflation, tax_rate, final_principle]
    
    variable_names = ['years_to_retire', 'how_much_needed_at_retirement', 'starting_principle',
                 'how_much_ss_at_retirement', 'how_much_other_passive_income', 'investment_apr',
                 'yrs_to_death', 'additional_principle', 'inflation', 'tax_rate', 
                 'final_principle']
    ''' 
    legend 
        variables[0] = years_to_retire 
        variables[1] = how_much_needed_at_retirement
        variables[2] = starting_principle
        variables[3] = how_much_ss_at_retirement
        variables[4] = how_much_other_passive_income
        variables[5] = investment_apr 
        variables[6] = yrs_to_death
        variables[7] = additional_principle
        variables[8] = inflation
        variables[9] = tax_rate
        variables[10] = final_principle
    '''
        
    return variables, variable_names

#initializes the dataframe
def init_df():
    
    variables, variable_names = var_input()
    df = pd.DataFrame(columns= [variable_names]) #creates the df using variable_names list
    
    return df, variables, variable_names

#determines the amount of gain over entirement retirement period (retirement to death)
def impact_years_compounding(variables, df):
    yr = 1
    starting_principle = variables[2]
    while yr <= variables[6]: #calculates compounding across multiple years
        if yr > variables[0] :#ends additional contribution to retirement at the start of retirement    
            additional_principle = 0
        else:
            additional_principle = variables[7]
          
        if yr < variables[0]: #years_to_retire: starts retirement debit at the start of retirement
            debit = 0
        else:
            debit = -variables[1] #how_much_needed_at_retirement
            
        final_principle, apr = annual_by_month_compound(variables, starting_principle, 
                                                        additional_principle, debit)
        
        '''df = df.append({'APR': apr, 'Year Number': yr, 'Starting Balance': starting_principle, 
                            'Ending Balance': final_principle, 'Additional Premium' : additional_premium, 
                            'Debit' : debit} , ignore_index = True)'''
        
        '''df = df.append({'years_to_retire': variables[0], 
                        'how_much_needed_at_retirement' : debit, 
                        'starting_principle': variables[2],
                        'how_much_ss_at_retirement': variables[3], 
                        'how_much_other_passive_income': variables[4], 
                        'investment_apr': variables[5],
                        'yrs_to_death': variables[6], 
                        'additional_principle': additional_principle, 
                        'inflation': variables[8], 
                        'tax_rate': variables[9], 
                        'final_principle': final_principle}, 
                        ignore_index = True)                '''
        
        df = df.append({variable_names[0]: variables[0], 
                        variable_names[1] : debit, 
                        variable_names[2]: variables[2],
                        variable_names[3]: variables[3], 
                        variable_names[4]: variables[4], 
                        variable_names[5]: variables[5],
                        variable_names[6]: variables[6], 
                        variable_names[7]: additional_principle, 
                        variable_names[8]: variables[8], 
                        variable_names[9]: variables[9], 
                        variable_names[10]: final_principle}, 
                        ignore_index = True)                
        
        starting_principle = final_principle
        
        yr += 1
        
    return df
    
#defines gain through monthly compounding while taking into account additional principle
def annual_by_month_compound(variables, starting_principle, additional_principle, debit): #interest rate compounded monthly for one year
    mpr = variables[5]/1200 # APR divided by 12 to get monthly interest rate
    #print('mpr = ', mpr)
    month = 1    
    new_principle = starting_principle
    #print('new_principle = ', new_principle, '\n')
    while month < 13:
        gain_from_interest_on_principle = new_principle * mpr
        #print('gain_from_interest_on_principle =', gain_from_interest_on_principle)
        new_principle = new_principle + gain_from_interest_on_principle + additional_principle/12 + debit/12
        month += 1
        new_principle = float(new_principle)
    return new_principle, variables[5]



In [19]:
#########################################
# MAIN #
#########################################

df, variables, variable_names = init_df()

df = impact_years_compounding(variables, df)

print(df)

In how many years do you plan to retire? 5
How much per year will you need at retirement? 50
How much have you saved for retirement right now? [in thousands]100
How much Social Security benefits per year will you get at retirement? 15
How much other passive income per year will you get at retirement? 12
What is the annualized percent return on your retirement saving before retiring? 15
How long will you need retirement income? [press enter if you want 20 yrs.]
Based on your input you will need your retirement income for  20.0 years.
How much will you be adding to your retirement savings per year before retiring? 12
What is the inflation rate per year? [press enter if you want 3%.]
Based on your input the inflation rate is 3.0 %.
Based on what you will need, what will be your tax rate? [press enter if you want to assume 25% for both state and federal taxes.]
Based on your input your overall income tax rate is 25.0 %.


TypeError: Expected tuple, got str

In [28]:
var_input()

In how many years do you plan to retire? 5
How much per year will you need at retirement? 5
How much have you saved for retirement right now? [in thousands]5
How much Social Security benefits will you get at retirement? 5
How much other passive income will you get at retirement? 5
What is the annualized percent return on your retirement saving before retiring? 5
How long will you need retirement income? [press enter if you want 20 yrs.]
Based on your input you will need your retirement income for  20.0 years.
How much will you be adding to your retirement savings per year before retiring? 0
What is the inflation rate? [press enter if you want 3%.]0
Based on your input the inflation rate is 3.0 %.


[5, 5.0, 5.0, '5', 5.0, 5.0, 20.0, 0.0, 3.0]

In [15]:

years_to_retire = 5
how_much_needed_at_retirement = 50
starting_principle = 100
how_much_ss_at_retirement = 15 
how_much_other_passive_income = 10
investment_apr = 15
yrs_to_death = 20
additional_principle = 12 
inflation = 3
tax_rate = 25

debit = 5
additional_principle = 2
final_principle = 3

variables = [years_to_retire, how_much_needed_at_retirement, starting_principle,
            how_much_ss_at_retirement, how_much_other_passive_income, investment_apr,
            yrs_to_death, additional_principle, inflation, tax_rate, final_principle]
    
variable_names = ['years_to_retire', 'how_much_needed_at_retirement', 'starting_principle',
                 'how_much_ss_at_retirement', 'how_much_other_passive_income', 'investment_apr',
                 'yrs_to_death', 'additional_principle', 'inflation', 'tax_rate', 
                 'final_principle']

df = pd.DataFrame(columns = variable_names)
    
df = df.append({'years_to_retire': variables[0], 
                        'how_much_needed_at_retirement' : debit, 
                        'starting_principle': variables[2],
                        'how_much_ss_at_retirement': variables[3], 
                        'how_much_other_passive_income': variables[4], 
                        'investment_apr': variables[5],
                        'yrs_to_death': variables[6], 
                        'additional_principle': additional_principle, 
                        'inflation': variables[8], 
                        'tax_rate': variables[9], 
                        'final_principle': final_principle}, 
                        ignore_index = True)    

display(df)

,years_to_retire,how_much_needed_at_retirement,starting_principle,how_much_ss_at_retirement,how_much_other_passive_income,investment_apr,yrs_to_death,additional_principle,inflation,tax_rate,final_principle
0,5,5,100,15,10,15,20,2,3,25,3


In [20]:
#inputs all of the needed information and return a list
def var_input():
    
    variables = []
    variable_names = []
    
    #time to retirement
    years_to_retire = int(input('In how many years do you plan to retire? '))
    
    #amount needed at retirement
    how_much_needed_at_retirement = float(input('How much per year will you need at retirement? '))
    
    #current retirement savings
    starting_principle = float(input('How much have you saved for retirement right now? [in thousands]'))
        
    #income after retirment
    how_much_ss_at_retirement = float(input('How much Social Security benefits per year will you get at retirement? '))
    how_much_other_passive_income = float(input('How much other passive income per year will you get at retirement? '))
    
    #return on retirement savings
    investment_apr = float(input('What is the annualized percent return on your retirement saving before retiring? '))

    #life expectance after retirement
    yrs_to_death = input('How long will you need retirement income? [press enter if you want 20 yrs.]')
    if yrs_to_death:
        yrs_to_death = float(yrs_to_death)
    else:
        yrs_to_death = float(20)
        print('Based on your input you will need your retirement income for ', yrs_to_death, 'years.')
    
    #contribution to retirement savings before retiring    
    additional_principle = float(input('How much will you be adding to your retirement savings per year before retiring? '))

    #inflation rate
    inflation = input('What is the inflation rate per year? [press enter if you want 3%.]')
    if inflation:
        inflation = float(inflation)
    else:
        inflation = float(3)
        print('Based on your input the inflation rate is', inflation, '%.')

    #tax rate
    tax_rate = input('Based on what you will need, what will be your tax rate? [press enter if you want to assume 25% for both state and federal taxes.]')
    if tax_rate:
        tax_rate = float(tax_rate)
    else:
        tax_rate = float(25)
        print('Based on your input your overall income tax rate is', tax_rate, '%.')
        
    final_principle = float(0)
    
    variables = [years_to_retire, how_much_needed_at_retirement, starting_principle,
                 how_much_ss_at_retirement, how_much_other_passive_income, investment_apr,
                 yrs_to_death, additional_principle, inflation, tax_rate, final_principle]
    
    variable_names = ['years_to_retire', 'how_much_needed_at_retirement', 'starting_principle',
                 'how_much_ss_at_retirement', 'how_much_other_passive_income', 'investment_apr',
                 'yrs_to_death', 'additional_principle', 'inflation', 'tax_rate', 
                 'final_principle']
    ''' 
    legend 
        variables[0] = years_to_retire 
        variables[1] = how_much_needed_at_retirement
        variables[2] = starting_principle
        variables[3] = how_much_ss_at_retirement
        variables[4] = how_much_other_passive_income
        variables[5] = investment_apr 
        variables[6] = yrs_to_death
        variables[7] = additional_principle
        variables[8] = inflation
        variables[9] = tax_rate
        variables[10] = final_principle
    '''
    return variables, variable_names

def init_df():
    
    variables, variable_names = var_input()
    df = pd.DataFrame(columns= [variable_names]) #creates the df using variable_names list
    
    return df, variables, variable_names

In [21]:

#defines gain through monthly compounding while taking into account additional principle
def annual_by_month_compound(variables, starting_principle, additional_principle, debit): #interest rate compounded monthly for one year
    mpr = variables[5]/1200 # APR divided by 12 to get monthly interest rate
    #print('mpr = ', mpr)
    month = 1    
    new_principle = starting_principle
    #print('new_principle = ', new_principle, '\n')
    while month < 13:
        gain_from_interest_on_principle = new_principle * mpr
        #print('gain_from_interest_on_principle =', gain_from_interest_on_principle)
        new_principle = new_principle + gain_from_interest_on_principle + additional_principle/12 + debit/12
        month += 1
        new_principle = float(new_principle)
    return new_principle, variables[5]

#determines the amount of gain over entirement retirement period (retirement to death)
def impact_years_compounding(variables, df):
    yr = 1
    starting_principle = variables[2]
    while yr <= variables[6]: #calculates compounding across multiple years
        if yr > variables[0] :#ends additional contribution to retirement at the start of retirement    
            additional_principle = 0
        else:
            additional_principle = variables[7]
          
        if yr < variables[0]: #years_to_retire: starts retirement debit at the start of retirement
            debit = 0
        else:
            debit = -variables[1] #how_much_needed_at_retirement
            
        final_principle, apr = annual_by_month_compound(variables, starting_principle, additional_principle, debit)
        
        #final_principle = 200
        df = df.append({variable_names[0]: variables[0], 
                        variable_names[1] : debit, 
                        variable_names[2]: variables[2],
                        variable_names[3]: variables[3], 
                        variable_names[4]: variables[4], 
                        variable_names[5]: variables[5],
                        variable_names[6]: variables[6], 
                        variable_names[7]: additional_principle, 
                        variable_names[8]: variables[8], 
                        variable_names[9]: variables[9], 
                        variable_names[10]: final_principle}, ignore_index = True)                
        
        starting_principle = final_principle
        
        yr += 1
        
    return df
    

df, variables, variable_names = init_df()
df = impact_years_compounding(variables, df)

In how many years do you plan to retire? 5
How much per year will you need at retirement? 50
How much have you saved for retirement right now? [in thousands]100
How much Social Security benefits per year will you get at retirement? 15
How much other passive income per year will you get at retirement? 12
What is the annualized percent return on your retirement saving before retiring? 15
How long will you need retirement income? [press enter if you want 20 yrs.]
Based on your input you will need your retirement income for  20.0 years.
How much will you be adding to your retirement savings per year before retiring? 12
What is the inflation rate per year? [press enter if you want 3%.]
Based on your input the inflation rate is 3.0 %.
Based on what you will need, what will be your tax rate? [press enter if you want to assume 25% for both state and federal taxes.]
Based on your input your overall income tax rate is 25.0 %.


TypeError: Expected tuple, got str

In [17]:
display(df)

,years_to_retire,how_much_needed_at_retirement,starting_principle,how_much_ss_at_retirement,how_much_other_passive_income,investment_apr,yrs_to_death,additional_principle,inflation,tax_rate,final_principle
0,5,5,100,15,10,15,20,2,3,25,3
1,5.0,0.0,100.0,15.0,10.0,15.0,20.0,2.0,3.0,25.0,118.218845
2,5.0,0.0,100.0,15.0,10.0,15.0,20.0,2.0,3.0,25.0,139.366452
3,5.0,0.0,100.0,15.0,10.0,15.0,20.0,2.0,3.0,25.0,163.913633
4,5.0,0.0,100.0,15.0,10.0,15.0,20.0,2.0,3.0,25.0,192.406883
5,5.0,-50.0,100.0,15.0,10.0,15.0,20.0,2.0,3.0,25.0,171.895713
6,5.0,-50.0,100.0,15.0,10.0,15.0,20.0,0.0,3.0,25.0,145.943887
7,5.0,-50.0,100.0,15.0,10.0,15.0,20.0,0.0,3.0,25.0,115.820187
8,5.0,-50.0,100.0,15.0,10.0,15.0,20.0,0.0,3.0,25.0,80.853966
9,5.0,-50.0,100.0,15.0,10.0,15.0,20.0,0.0,3.0,25.0,40.266767


In [13]:
#defines gain through monthly compounding while taking into account additional principle
def annual_by_month_compound(variables, starting_principle, additional_principle, debit): #interest rate compounded monthly for one year
    mpr = variables[5]/1200 # APR divided by 12 to get monthly interest rate
    #print('mpr = ', mpr)
    month = 1    
    new_principle = starting_principle
    #print('new_principle = ', new_principle, '\n')
    while month < 13:
        gain_from_interest_on_principle = new_principle * mpr
        #print('gain_from_interest_on_principle =', gain_from_interest_on_principle)
        new_principle = new_principle + gain_from_interest_on_principle + additional_principle/12 + debit/12
        month += 1
        new_principle = float(new_principle)
    return new_principle, variables[5]



In [5]:
def var_input():
    
    variables = []
    variable_names = []
    
    #time to retirement
    years_to_retire = int(input('In how many years do you plan to retire? '))
    
    #amount needed at retirement
    how_much_needed_at_retirement = float(input('How much per year will you need at retirement? '))
    
    #current retirement savings
    starting_principle = float(input('How much have you saved for retirement right now? [in thousands]'))
        
    #income after retirment
    how_much_ss_at_retirement = float(input('How much Social Security benefits per year will you get at retirement? '))
    how_much_other_passive_income = float(input('How much other passive income per year will you get at retirement? '))
    
    #return on retirement savings
    investment_apr = float(input('What is the annualized percent return on your retirement saving before retiring? '))

    #life expectance after retirement
    yrs_to_death = input('How long will you need retirement income? [press enter if you want 20 yrs.]')
    if yrs_to_death:
        yrs_to_death = float(yrs_to_death)
    else:
        yrs_to_death = float(20)
        print('Based on your input you will need your retirement income for ', yrs_to_death, 'years.')
    
    #contribution to retirement savings before retiring    
    additional_principle = float(input('How much will you be adding to your retirement savings per year before retiring? '))

    #inflation rate
    inflation = input('What is the inflation rate per year? [press enter if you want 3%.]')
    if inflation:
        inflation = float(inflation)
    else:
        inflation = float(3)
        print('Based on your input the inflation rate is', inflation, '%.')

    #tax rate
    tax_rate = input('Based on what you will need, what will be your tax rate? [press enter if you want to assume 25% for both state and federal taxes.]')
    if tax_rate:
        tax_rate = float(tax_rate)
    else:
        tax_rate = float(25)
        print('Based on your input your overall income tax rate is', tax_rate, '%.')
        

    final_principle = float(0)
    
    variables = [years_to_retire, how_much_needed_at_retirement, starting_principle,
                 how_much_ss_at_retirement, how_much_other_passive_income, investment_apr,
                 yrs_to_death, additional_principle, inflation, tax_rate, final_principle]
    
    variable_names = ['years_to_retire', 'how_much_needed_at_retirement', 'starting_principle',
                 'how_much_ss_at_retirement', 'how_much_other_passive_income', 'investment_apr',
                 'yrs_to_death', 'additional_principle', 'inflation', 'tax_rate', 
                 'final_principle']
    ''' 
    legend 
        variables[0] = years_to_retire 
        variables[1] = how_much_needed_at_retirement
        variables[2] = starting_principle
        variables[3] = how_much_ss_at_retirement
        variables[4] = how_much_other_passive_income
        variables[5] = investment_apr 
        variables[6] = yrs_to_death
        variables[7] = additional_principle
        variables[8] = inflation
        variables[9] = tax_rate
        variables[10] = final_principle
    '''
        
    return variables, variable_names


In [6]:
var, var_name = var_input()

In how many years do you plan to retire? 5
How much per year will you need at retirement? 50
How much have you saved for retirement right now? [in thousands]100
How much Social Security benefits per year will you get at retirement? 15
How much other passive income per year will you get at retirement? 12
What is the annualized percent return on your retirement saving before retiring? 15
How long will you need retirement income? [press enter if you want 20 yrs.]
Based on your input you will need your retirement income for  20.0 years.
How much will you be adding to your retirement savings per year before retiring? 12
What is the inflation rate per year? [press enter if you want 3%.]
Based on your input the inflation rate is 3.0 %.
Based on what you will need, what will be your tax rate? [press enter if you want to assume 25% for both state and federal taxes.]
Based on your input your overall income tax rate is 25.0 %.


In [7]:
display(var)

[5, 50.0, 100.0, 15.0, 12.0, 15.0, 20.0, 12.0, 3.0, 25.0, 0.0]

In [8]:
display(var_name)

['years_to_retire',
 'how_much_needed_at_retirement',
 'starting_principle',
 'how_much_ss_at_retirement',
 'how_much_other_passive_income',
 'investment_apr',
 'yrs_to_death',
 'additional_principle',
 'inflation',
 'tax_rate',
 'final_principle']

In [ ]:
df = pd.DataFrame(columns= ['APR', 'Years', 'Starting Balance', 'Ending Balance'])
list(df.columns)

In [ ]:
i = 0

while i < 5:
        
    APR = i
    Years = i
    S = i
    B = i 
    
    df = df.append({'APR': APR, 'Years': Years, 'Starting Balance': S, 'Ending Balance': B} , ignore_index = True)

    i += 1
        
print(df)

In [ ]:
print(df)

In [ ]:
df = pd.DataFrame(columns=['A'])
for i in range(5):
    df = df.append({'A': i}, ignore_index=True)
df